In [11]:
TARGET_LOCATION = 'Plymouth, MA'
TARGET_LOCATION = 'Jamaica Plain, MA'
TARGET_LOCATION = 'Andover, MA'
TARGET_LOCATION = 'Lynn, MA'
TARGET_LOCATION = 'Chelmsford, MA'
# TARGET_LOCATION = 'Boulder, CO'
TARGET_LOCATION = 'Lafayette, CO'

from model_lib import RedfinModel

In [12]:
redfin = RedfinModel(TARGET_LOCATION, {
  'style': ['SINGLE_FAMILY', 'TOWNHOUSE'],
  'beds': [3, 4, 5, 6],
  'full_baths': [2, 3, 4],
})
train_df = redfin.fetch_data('sold')
#
train_df.head(3)

Using cached data for Lafayette, CO sold as of 2024-08-05
filtering column: style allowed_values: ['SINGLE_FAMILY', 'TOWNHOUSE']
filtering column: beds allowed_values: [3, 4, 5, 6]
filtering column: full_baths allowed_values: [2, 3, 4]
Filtered data shape: (111, 44) (from (363, 43))


,property_url,mls,mls_id,status,text,style,full_street_line,street,unit,city,...,agent,agent_email,agent_phones,broker,broker_phone,broker_website,nearby_schools,primary_photo,alt_photos,age
9,https://www.realtor.com/realestateandhomes-det...,DVCO,8832188,SOLD,"BRAND NEW HOME!! Introducing the Camellia, a v...",SINGLE_FAMILY,1300 Brookfield Pl,1300 Brookfield Pl,<NA>,Erie,...,TEAM KAMINSKY/ DOELL,team@landmarkcolorado.com,"[{'number': '7202487653', 'type': 'Mobile', 'e...",<NA>,<NA>,<NA>,Boulder Valley Re 2 School District,http://ap.rdcpix.com/377f6f3015b60a6613fd1636c...,http://ap.rdcpix.com/377f6f3015b60a6613fd1636c...,0
13,https://www.realtor.com/realestateandhomes-det...,DVCO,8436533,SOLD,Welcome to your new home in the desirable Indi...,SINGLE_FAMILY,2468 Lexington St,2468 Lexington St,<NA>,Lafayette,...,Ben Smith,ben@westandmainhomes.com,<NA>,<NA>,<NA>,<NA>,Boulder Valley Re 2 School District,http://ap.rdcpix.com/70e403b45a7b83aada0cd6c73...,http://ap.rdcpix.com/70e403b45a7b83aada0cd6c73...,26
14,https://www.realtor.com/realestateandhomes-det...,DVCO,5573340,SOLD,Discover the perfect blend of lifestyle and co...,SINGLE_FAMILY,270 N Cherrywood Dr,270 N Cherrywood Dr,<NA>,Lafayette,...,Ryan Prast,ryanprast@Listings.com,"[{'number': '7207323498', 'type': 'Mobile', 'e...",<NA>,<NA>,<NA>,Boulder Valley Re 2 School District,http://ap.rdcpix.com/cf27d8fb297605ca908b6b51a...,http://ap.rdcpix.com/cf27d8fb297605ca908b6b51a...,30


In [13]:
train_df.columns.values
# one hot encode columns

array(['property_url', 'mls', 'mls_id', 'status', 'text', 'style',
       'full_street_line', 'street', 'unit', 'city', 'state', 'zip_code',
       'beds', 'full_baths', 'half_baths', 'sqft', 'year_built',
       'days_on_mls', 'list_price', 'list_date', 'sold_price',
       'last_sold_date', 'assessed_value', 'estimated_value', 'lot_sqft',
       'price_per_sqft', 'latitude', 'longitude', 'neighborhoods',
       'county', 'fips_code', 'stories', 'hoa_fee', 'parking_garage',
       'agent', 'agent_email', 'agent_phones', 'broker', 'broker_phone',
       'broker_website', 'nearby_schools', 'primary_photo', 'alt_photos',
       'age'], dtype=object)

In [14]:
redfin.train_from_raw(train_df, train_df[RedfinModel.TARGET_COLUMN], show_debug=True)
print(redfin.trained_columns)


Using columns: ['age' 'beds' 'days_on_mls' 'fips_code' 'full_baths' 'half_baths'
 'hoa_fee' 'lot_sqft' 'parking_garage' 'sqft' 'stories' 'unit' 'city=Erie'
 'city=Lafayette' 'nearby_schools'
 'nearby_schools=Boulder Valley Re 2 School District' 'state=CO'
 'style=SINGLE_FAMILY']
Processed data shape: (111, 18)
Processed data columns: 18
['age' 'beds' 'days_on_mls' 'fips_code' 'full_baths' 'half_baths'
 'hoa_fee' 'lot_sqft' 'parking_garage' 'sqft' 'stories' 'unit' 'city=Erie'
 'city=Lafayette' 'nearby_schools'
 'nearby_schools=Boulder Valley Re 2 School District' 'state=CO'
 'style=SINGLE_FAMILY']


In [15]:
mean_score = redfin.show_cross_validation(train_df.drop(RedfinModel.TARGET_COLUMN, axis=1), train_df[RedfinModel.TARGET_COLUMN])


Cross-validation scores: [0.55579902 0.45259488 0.53597762 0.58836682 0.63400879]
Average cross-validation score: 0.55


In [16]:
test_df = redfin.fetch_data('for_sale')
# test_df = redfin.filter_data(test_df)
result_df = redfin.predict(test_df)

Using cached data for Lafayette, CO for_sale as of 2024-08-05
filtering column: style allowed_values: ['SINGLE_FAMILY', 'TOWNHOUSE']
filtering column: beds allowed_values: [3, 4, 5, 6]
filtering column: full_baths allowed_values: [2, 3, 4]
Filtered data shape: (65, 44) (from (148, 43))
Dropping columns: ['city=Louisville' 'nearby_schools=St Vrain Valley Re 1j School District']
Adding columns: ['nearby_schools']
Predicted 65 values


In [17]:
print(redfin.trained_columns)
# remove exponent formatting
result_df[RedfinModel.OUTPUT_COLUMNS].sort_values(by=['diff_percent'], ascending=False).head(25)


['age' 'beds' 'days_on_mls' 'fips_code' 'full_baths' 'half_baths'
 'hoa_fee' 'lot_sqft' 'parking_garage' 'sqft' 'stories' 'unit' 'city=Erie'
 'city=Lafayette' 'nearby_schools'
 'nearby_schools=Boulder Valley Re 2 School District' 'state=CO'
 'style=SINGLE_FAMILY']


,readable_address,style,sqft,beds,full_baths,list_price,predicted,diff,diff_percent,property_url
74,"1740 Lyonesse St, Lafayette, CO",SINGLE_FAMILY,1824,4,2,540000,713978.80,173978.8,32.218296,https://www.realtor.com/realestateandhomes-det...
103,"1905 Shallot Cir, Lafayette, CO",SINGLE_FAMILY,1165,4,2,650000,811615.80,161615.8,24.863969,https://www.realtor.com/realestateandhomes-det...
89,"2946 Golden Eagle Cir, Lafayette, CO",SINGLE_FAMILY,3472,5,2,875000,1073659.50,198659.5,22.703943,https://www.realtor.com/realestateandhomes-det...
66,"1255 Doric Dr, Lafayette, CO",SINGLE_FAMILY,1900,4,2,650000,797322.24,147322.24,22.66496,https://www.realtor.com/realestateandhomes-det...
9,"614 Glenwood Dr, Lafayette, CO",SINGLE_FAMILY,1704,4,2,675000,817030.76,142030.76,21.041594,https://www.realtor.com/realestateandhomes-det...
18,"614 Glenwood Dr, Lafayette, CO",SINGLE_FAMILY,1704,4,2,675000,814792.12,139792.12,20.709944,https://www.realtor.com/realestateandhomes-det...
108,"1760 Shallot Cir, Lafayette, CO",SINGLE_FAMILY,1664,4,2,599900,722236.94,122336.94,20.392889,https://www.realtor.com/realestateandhomes-det...
137,"302 E Chester St, Lafayette, CO",SINGLE_FAMILY,1970,5,3,689000,804303.70,115303.7,16.734935,https://www.realtor.com/realestateandhomes-det...
44,"447 Pinewood Cir, Lafayette, CO",SINGLE_FAMILY,2100,4,2,695000,790745.80,95745.8,13.776374,https://www.realtor.com/realestateandhomes-det...
59,"2365 Cedarwood Cir, Lafayette, CO",SINGLE_FAMILY,2993,5,2,800000,897169.00,97169.0,12.146125,https://www.realtor.com/realestateandhomes-det...


In [18]:
importances = redfin.print_feature_importances()
print(f"===\nFeature Importances ({len(importances)})\n===")
for importance in importances:
    print(importance)



===
Feature Importances (18)
===
('lot_sqft', np.float64(0.26266623765363684))
('sqft', np.float64(0.23636092385747667))
('hoa_fee', np.float64(0.146096435695211))
('age', np.float64(0.11797574220792562))
('days_on_mls', np.float64(0.06673926296164416))
('beds', np.float64(0.06532599853807217))
('parking_garage', np.float64(0.02844837892526952))
('full_baths', np.float64(0.02686932590437269))
('city=Erie', np.float64(0.015172296738853704))
('half_baths', np.float64(0.013362081488280254))
('city=Lafayette', np.float64(0.009723478054921512))
('stories', np.float64(0.009514298384073307))
('fips_code', np.float64(0.0013131097753083533))
('nearby_schools=Boulder Valley Re 2 School District', np.float64(0.0004324298149541617))
('unit', np.float64(0.0))
('nearby_schools', np.float64(0.0))
('state=CO', np.float64(0.0))
('style=SINGLE_FAMILY', np.float64(0.0))


In [19]:
# test_df.head()
print(set(result_df['style']))

{'SINGLE_FAMILY'}


In [20]:
result_df[RedfinModel.OUTPUT_COLUMNS].to_dict(orient='records')[0]

{'readable_address': '2975 Thunder Lake Cir, Lafayette, CO',
 'style': 'SINGLE_FAMILY',
 'sqft': 4195,
 'beds': 6,
 'full_baths': 3,
 'list_price': 1295000,
 'predicted': 1062071.3,
 'diff': -232928.69999999995,
 'diff_percent': -17.986772200772197,
 'property_url': 'https://www.realtor.com/realestateandhomes-detail/2319385537'}